In [63]:
import torch
import torch.optim as optim
import numpy as np
import random
import glob
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch.nn as nn
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.utils import save_image 
from PIL import Image
import os

In [77]:
class GarbageDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths
        self.labels = {"cardboard": 0,
          "glass": 1,
          "metal": 2,
          "paper": 3,
          "plastic": 4,
          "trash": 5}

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        x = self.paths[index]
        label = x.split("\\")[-2]
        label = self.labels[label]
    
        img = Image.open(x)
        img = img.resize((256,256))
        img = np.array(img)
        img = np.reshape(img,(3,256,256))
        
        return torch.tensor(img,dtype= torch.float32), torch.tensor(label,dtype= torch.float32)

In [ ]:
import os
path = ("Garbage classification")

if not os.path.exists(path):
    print("Belirtilen yol mevcut değil!")

In [79]:
all_path = glob.glob("C:\\Users\\Sisasoft\\Desktop\\GarbageProject\\Garbage classification\\*\\*")

In [80]:
while len(all_path)<=15000:
    random.shuffle(all_path)
    for i in range(len(all_path)):
        x = all_path[i]
        label = x.split("\\")[-2]
        
        if i<= int((len(all_path))/ 3):
            images = Image.open(all_path[i])
            augmented_images = transforms.RandomHorizontalFlip()(images)
            
        elif (i> int((len(all_path))/ 3)) and (i<= 2*int((len(all_path))/ 3)):
            images = Image.open(all_path[i])
            augmented_images = transforms.RandomRotation(degrees=60)(images)

        else:
            images = Image.open(all_path[i])
            augmented_images = transforms.ColorJitter()(images)
    
        
        
        if label == "cardboard":
            save_path = os.path.join("Garbage classification\cardboard", f"newcardboard{i}.jpg")
            augmented_images.save(save_path)
        elif label == "glass":
            save_path = os.path.join("Garbage classification\glass", f"newglass{i}.jpg")
            augmented_images.save(save_path)
        elif label == "metal":
            save_path = os.path.join("Garbage classification\metal", f"newmetal{i}.jpg")
            augmented_images.save(save_path)   
        elif label == "paper":
            save_path = os.path.join("Garbage classification\paper", f"newpaper{i}.jpg")
            augmented_images.save(save_path)
        elif label == "plastic":
            save_path = os.path.join("Garbage classification\plastic", f"newcplastic{i}.jpg")
            augmented_images.save(save_path)
        if label == "trash":
            save_path = os.path.join("Garbage classification\\trash", f"newtrash{i}.jpg")
            augmented_images.save(save_path)

In [81]:
all_path = glob.glob("C:\\Users\\Sisasoft\\Desktop\\GarbageProject\\Garbage classification\\*\\*")

In [82]:
len(all_path)

15080

In [69]:
random.shuffle(all_path)
train_set = all_path[:int(len(all_path)*0.8)]
test_set = all_path[int(len(all_path)*0.8):]
train_dataset = GarbageDataset(train_set)
test_dataset = GarbageDataset(test_set)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [70]:
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel,self).__init__()
        self.relu = nn.ReLU()
        self.conv1= nn.Conv2d(3,6,5)
        self.pool=nn.MaxPool2d(4,4)
        self.conv2=nn.Conv2d(6, 16, 5)
        self.pool2=nn.MaxPool2d(4, 4)
        #self.drop = nn.Dropout2d(0.2)
        self.conv3=nn.Conv2d(16, 56, 5)
        self.pool3=nn.MaxPool2d(4, 4)
        self.flatten=nn.Flatten()
        self.fc1=nn.Linear(56 *2 *2, 120)
        self.fc2=nn.Linear(120, 84)
        self.fc3=nn.Linear(84, 6)
        self.sigmoid= nn.Sigmoid()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        #print(x.size())
        x = self.pool2(F.relu(self.conv2(x)))
        #print(x.size())
        #x = F.dropout(self.drop(x), training=self.training)
        x = self.pool3(F.relu(self.conv3(x)))
        #print(x.size())
        x = self.flatten(x)
        #print(x.size())
        x = F.relu(self.fc1(x))
        #print(x.size())
        x = F.relu(self.fc2(x))
        #print(x.size())
        x = self.fc3(x)
        return x

In [71]:
model = CNNModel()
model.load_state_dict(torch.load('model.pth'))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [76]:
epochs= 5 () #variable by model

for epoch in range(epochs):

    model.train()
    loss = 0
    for i, data in enumerate(train_loader,0):
        inputs, labels = data
        outputs = model(inputs)
        #print(labels)       
        train_loss= criterion(outputs,labels.type(torch.LongTensor))
        optimizer.zero_grad()
        train_loss.backward() #ağırlıkları günceller
        optimizer.step() 
        loss += train_loss.item()
    loss = loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}:",f"loss = {loss :.3f}")

Epoch 1/5: loss = 0.591
Epoch 2/5: loss = 0.549
Epoch 3/5: loss = 0.511
Epoch 4/5: loss = 0.472
Epoch 5/5: loss = 0.452


In [74]:
torch.save(model.state_dict(), 'model.pth')

In [83]:
model.eval()
loss2 = 0
accuracy = 0
total = 0

with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs, labels = data
        outputs = model(inputs)
        test_loss = criterion(outputs,labels.type(torch.LongTensor))
        loss2 += test_loss.item()
        _, predicted = torch.max(outputs, 1)
       
        accuracy += (predicted.type(torch.LongTensor) == labels).sum().item()


loss2 /= (len(test_loader))

accuracy /=  (len(test_loader)* 32)

print(f"Test loss = {loss2:.3f}")
print(f"Toplam test görüntüsü için doğruluk yüzdesi: {100*accuracy: .3f} %")

Test loss = 1.008
Toplam test görüntüsü için doğruluk yüzdesi:  72.566 %
